In [ ]:
!pip install darts

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.utils.statistics import plot_acf, plot_pacf
from darts.models import ARIMA
import matplotlib.pyplot as plt

In [3]:
tesla_data = yf.download('TSLA', start='2014-01-01', end='2023-11-01', interval = '1mo', progress=False)
tesla_data = tesla_data['Close']  # Nos enfocamos en los precios de cierre

# Aplicando el logaritmo usando NumPy
tesla_log_data = np.log(tesla_data)

# Convirtiendo de nuevo a TimeSeries de Darts
tesla_log_ts = TimeSeries.from_series(tesla_log_data, fill_missing_dates=True)

# Diferenciando la serie logarítmica
tesla_diff = tesla_log_ts.diff()


In [4]:
import pmdarima as pm

# Convierte la serie temporal de Darts a una serie de pandas
tesla_diff_pd = tesla_diff.pd_series()

# Ajusta el modelo ARIMA utilizando auto_arima
auto_model = pm.auto_arima(tesla_diff_pd, seasonal=False, stepwise=False,
                           suppress_warnings=True,
                           error_action="ignore",
                           max_order=None,
                           trace=True,
                           start_p =0 , max_p = 4,
                           start_q=0 , max_q=4)

# Obtén el AIC del modelo ajustado
aic_value_auto = auto_model.aic()
print(f'AIC auto modelo: {aic_value_auto}')

 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-86.433, Time=0.09 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=-84.742, Time=0.04 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=-82.982, Time=0.06 sec
 ARIMA(0,0,3)(0,0,0)[0]             : AIC=-81.038, Time=0.08 sec
 ARIMA(0,0,4)(0,0,0)[0]             : AIC=-92.346, Time=0.14 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=-84.704, Time=0.08 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=-83.891, Time=0.28 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=-80.984, Time=0.19 sec
 ARIMA(1,0,3)(0,0,0)[0]             : AIC=-84.876, Time=0.21 sec
 ARIMA(1,0,4)(0,0,0)[0]             : AIC=-90.399, Time=0.19 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=-83.140, Time=0.06 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=-81.146, Time=0.10 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=-82.009, Time=0.62 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=-90.595, Time=0.79 sec
 ARIMA(2,0,4)(0,0,0)[0]             : AIC=-89.425, Time=0.88 sec
 ARIMA(3,0,0)(0,0,0)[0]  

In [5]:
# Obtén los parámetros estimados del modelo seleccionado
estimated_params = auto_model.arima_res_.params
print(f'Parámetros estimados: {estimated_params}')

Parámetros estimados: ma.L1     0.068013
ma.L2    -0.017649
ma.L3    -0.051494
ma.L4     0.355978
sigma2    0.024292
dtype: float64


In [6]:
auto_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  117
Model:               SARIMAX(0, 0, 4)   Log Likelihood                  51.173
Date:                Thu, 16 Nov 2023   AIC                            -92.346
Time:                        01:00:09   BIC                            -78.535
Sample:                    02-01-2014   HQIC                           -86.739
                         - 10-01-2023                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.0680      0.067      1.012      0.312      -0.064       0.200
ma.L2         -0.0176      0.087     -0.204      0.839      -0.188       0.152
ma.L3         -0.0515      0.087     -0.593      0.553      -0.222       0.119
ma.L4          0.3560      0.086      4.134      0.000       0.187       0.525
sigma2         0.0243      0.003      7.420      0.000       0.018       0.031
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 1.04
Prob(Q):                              0.96   Prob(JB):                         0.59
Heteroskedasticity (H):               2.33   Skew:                             0.23
Prob(H) (two-sided):                  0.01   Kurtosis:                         3.08
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""